In [6]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import re
from io import StringIO

In [17]:
import pandas as pd
import requests
from io import StringIO

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}
response = requests.get(url, headers=headers)
sp500_table['CIK'] = sp500_table['CIK'].astype(str).str.zfill(10)
print(sp500_table['CIK'])

0      0000066740
1      0000091142
2      0000001800
3      0001551152
4      0001467373
          ...    
498    0001524472
499    0001041061
500    0000877212
501    0001136869
502    0001555280
Name: CIK, Length: 503, dtype: object


In [ ]:
earningsDF = pd.DataFrame(columns=['tickers', 'companyName', 'accessionNumber', 'document', 'filingDate', 'url'])
j = 0
for cik in sp500_table['CIK']:
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    user = ''
    request = requests.get(url, headers = {"User-Agent":user})
    forms = request.json()["filings"]["recent"]['form']
    primaryDocs = request.json()["filings"]["recent"]["primaryDocument"]
    accessionNumber = request.json()["filings"]["recent"]["accessionNumber"]
    filingDate = request.json()["filings"]["recent"]["filingDate"]
    companyName = request.json()['name']
    tickers = request.json()['tickers']
    for i in range(len(forms)):
        if forms[i] == '10-Q':
            url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accessionNumber[i].replace('-', '')}/{primaryDocs[i]}"
            earningsDF.loc[len(earningsDF)] = [tickers, companyName, accessionNumber[i], primaryDocs[i], filingDate[i], url]
    j += 1
    print(f"{j} out of 503, cik# {cik} finished.")
    time.sleep(0.1)
        

1 out of 503, cik# 0000066740 finished.
2 out of 503, cik# 0000091142 finished.
3 out of 503, cik# 0000001800 finished.
4 out of 503, cik# 0001551152 finished.
5 out of 503, cik# 0001467373 finished.
6 out of 503, cik# 0000796343 finished.
7 out of 503, cik# 0000002488 finished.
8 out of 503, cik# 0000874761 finished.
9 out of 503, cik# 0000004977 finished.
10 out of 503, cik# 0001090872 finished.
11 out of 503, cik# 0000002969 finished.
12 out of 503, cik# 0001559720 finished.
13 out of 503, cik# 0001086222 finished.
14 out of 503, cik# 0000915913 finished.
15 out of 503, cik# 0001035443 finished.
16 out of 503, cik# 0001097149 finished.
17 out of 503, cik# 0001579241 finished.
18 out of 503, cik# 0000352541 finished.
19 out of 503, cik# 0000899051 finished.
20 out of 503, cik# 0001652044 finished.
21 out of 503, cik# 0001652044 finished.
22 out of 503, cik# 0000764180 finished.
23 out of 503, cik# 0001018724 finished.
24 out of 503, cik# 0001748790 finished.
25 out of 503, cik# 00010

In [33]:
earningsDF.iloc[784]['url']

'https://www.sec.gov/Archives/edgar/data/0001410636/000156459014001874/awk-10q_20140331.htm'

In [34]:
earningsDF.to_pickle('10q_docs.pkl')

In [35]:
earningsDF

,tickers,companyName,accessionNumber,document,filingDate,url
0,[MMM],3M CO,0000066740-25-000089,mmm-20250930.htm,2025-10-21,https://www.sec.gov/Archives/edgar/data/000006...
1,[MMM],3M CO,0000066740-25-000063,mmm-20250630.htm,2025-07-18,https://www.sec.gov/Archives/edgar/data/000006...
2,[MMM],3M CO,0000066740-25-000039,mmm-20250331.htm,2025-04-22,https://www.sec.gov/Archives/edgar/data/000006...
3,[MMM],3M CO,0000066740-24-000101,mmm-20240930.htm,2024-10-22,https://www.sec.gov/Archives/edgar/data/000006...
4,[MMM],3M CO,0000066740-24-000080,mmm-20240630.htm,2024-07-26,https://www.sec.gov/Archives/edgar/data/000006...
...,...,...,...,...,...,...
13690,[ZTS],Zoetis Inc.,0001555280-20-000261,zts-20200630.htm,2020-08-06,https://www.sec.gov/Archives/edgar/data/000155...
13691,[ZTS],Zoetis Inc.,0001555280-20-000199,zts-20200331.htm,2020-05-06,https://www.sec.gov/Archives/edgar/data/000155...
13692,[ZTS],Zoetis Inc.,0001555280-19-000270,a2019q3form10-q.htm,2019-11-07,https://www.sec.gov/Archives/edgar/data/000155...
13693,[ZTS],Zoetis Inc.,0001555280-19-000221,a2019q2form10-q.htm,2019-08-06,https://www.sec.gov/Archives/edgar/data/000155...


In [ ]:
user = ''

def extract_mda_section(url, user_agent):
    """
    Extract the Management's Discussion and Analysis section from a 10-Q.
    """
    time.sleep(0.1)
    
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for element in soup(["script", "style"]):
        element.decompose()
    text = soup.get_text()
    
    pattern = r'Item\s*2[\.:]\s*Management[\'\’]s\s+Discussion\s+and\s+Analysis\s+of\s+Financial\s+Condition\s+and\s+Results\s+of\s+Operations'
    
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    
    if len(matches) >= 2:
        second_match = matches[1]
        start_pos = second_match.start()
        
        text_before = text[:start_pos]
        actual_start_pos = start_pos
        
        next_item_pattern = r'ITEM\s*[3][.:]'
        next_match = re.search(next_item_pattern, text[start_pos + 100:], re.IGNORECASE)
        
        if next_match:
            end_pos = start_pos + 100 + next_match.start()
            mda_text = text[actual_start_pos:end_pos]
        else:
            mda_text = text[actual_start_pos:start_pos + 50000]
        
        return mda_text
    elif len(matches) == 1:
        return "Only one instance of Item 2 MD&A found (expected second instance)"
    else:
        return "MD&A section not found"


test_extract = extract_mda_section(earningsDF.iloc[0]['url'], user)

In [46]:
# test_df = earningsDF.head(10).copy()
# test_df['content'] = None

# for index, row in test_df.iterrows():
#     try:
#         test_df.at[index, 'content'] = extract_mda_section(row['url'], user)
#         print(f"Processed row {index}")
#     except Exception as e:
#         test_df.at[index, 'content'] = f"Error: {str(e)}"

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9


In [53]:
earningsDF['content'] = None

for index, row in earningsDF.iterrows():
    try:
        earningsDF.at[index, 'content'] = extract_mda_section(row['url'], user)
        print(f"Processed row {index}")
    except Exception as e:
        earningsDF.at[index, 'content'] = f"Error: {str(e)}"

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Process

In [54]:
earningsDF.to_pickle('10q_text.pkl')

In [60]:
earningsDF.iloc[420]['content']

'ITEM 2.MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS. In addition to historical information, this quarterly report on Form\xa010-Q contains forward-looking statements within the meaning of Section\xa027A of the Securities Act of 1933 and Section\xa021E of the Securities Exchange Act of 1934. These statements include, among other things, our expectations regarding the anticipated impact of our new products and product enhancements will have on doctor utilization and our market share, our expectations regarding product mix and product adoption, our expectations regarding the existence and impact of seasonality, our expectations regarding the sales growth of our intra-oral scanner sales in international markets, our expectations regarding the financial and strategic benefits of establishing regional order acquisition, treatment planning and manufacturing facilities, as well as the anticipated timing of such facilities being operational, our expecta

In [59]:
earningsDF.iloc[420]

tickers                                                       [ALGN]
companyName                                     ALIGN TECHNOLOGY INC
accessionNumber                                 0001097149-18-000029
document                                           algn-20180630.htm
filingDate                                                2018-08-02
url                https://www.sec.gov/Archives/edgar/data/000109...
content            ITEM 2.MANAGEMENT’S DISCUSSION AND ANALYSIS OF...
Name: 420, dtype: object

In [ ]:
earnings